<a href="https://colab.research.google.com/github/apriandito/nowcasting/blob/main/7_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install streamlit pyngrok --quiet

In [5]:
%%writefile app.py
import streamlit as st
import warnings
warnings.filterwarnings("ignore")

import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import shap
import matplotlib.pyplot as plt
import plotly.graph_objects as go

st.set_page_config(layout="wide")
st.title("Prediksi USD/IDR Menggunakan Model Machine Learning")

API_KEY = '8V7F020H9WXQRHLQ'
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol=IDR&apikey={API_KEY}&outputsize=full"
response = requests.get(url)
data = response.json()
timeseries = data.get("Time Series FX (Daily)", {})

df = pd.DataFrame.from_dict(timeseries, orient='index')
df.columns = ['Open', 'High', 'Low', 'Close']
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df = df.astype(float)

df_feat = df.copy()
df_feat['Open_today'] = df_feat['Open']
df_feat['Close_t-1'] = df_feat['Close'].shift(1)
df_feat['Open_t-1'] = df_feat['Open'].shift(1)
df_feat['High_t-1'] = df_feat['High'].shift(1)
df_feat['Low_t-1'] = df_feat['Low'].shift(1)
df_feat['MA_3'] = df_feat['Close'].shift(1).rolling(window=3).mean()
df_feat['MA_5'] = df_feat['Close'].shift(1).rolling(window=5).mean()
df_feat['STD_3'] = df_feat['Close'].shift(1).rolling(window=3).std()
df_feat['STD_5'] = df_feat['Close'].shift(1).rolling(window=5).std()
df_feat['ROC_1'] = df_feat['Close'].shift(1).pct_change(periods=1)
df_feat['ROC_3'] = df_feat['Close'].shift(1).pct_change(periods=3)
df_feat['Momentum_1'] = df_feat['Close'].shift(1) - df_feat['Close'].shift(2)
df_feat['Momentum_3'] = df_feat['Close'].shift(1) - df_feat['Close'].shift(4)
df_feat.dropna(inplace=True)

latest_row = df_feat.iloc[[-1]]
df_feat = df_feat.iloc[:-1]

X = df_feat[[
    'Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1',
    'MA_3', 'MA_5', 'STD_3', 'STD_5', 'ROC_1', 'ROC_3', 'Momentum_1', 'Momentum_3'
]]
y = df_feat['Close']

models = {
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.01),
    'BayesianRidge': BayesianRidge(),
}

validation_results = []
start_index = 60
for t in range(start_index, len(X)):
    X_train, y_train = X.iloc[:t], y.iloc[:t]
    X_test, y_test_point = X.iloc[t:t+1], y.iloc[t:t+1]
    result = {
        'Date': y_test_point.index[0],
        'Actual': y_test_point.values[0]
    }
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)[0]
        result[f'{name}_Pred'] = y_pred
    validation_results.append(result)

val_df = pd.DataFrame(validation_results).set_index('Date')

# Visualisasi
fig = go.Figure()
fig.add_trace(go.Scatter(x=val_df.index, y=val_df['Actual'], mode='lines', name='Actual'))

for name in models.keys():
    fig.add_trace(go.Scatter(x=val_df.index, y=val_df[f'{name}_Pred'], mode='lines', name=name))

st.plotly_chart(fig, use_container_width=True)

best_model_name = 'BayesianRidge'
best_model = models[best_model_name]
best_model.fit(X, y)

X_latest = latest_row[X.columns]
y_actual = latest_row['Close'].values[0]
y_forecast = best_model.predict(X_latest)[0]

st.subheader("Prediksi Hari Terakhir")
st.write(f"**Tanggal:** {latest_row.index[0].date()}")
st.write(f"**Actual Close:** {y_actual:.2f}")
st.write(f"**Predicted Close ({best_model_name}):** {y_forecast:.2f}")
st.write(f"**Absolute Error:** {abs(y_forecast - y_actual):.2f}")

Overwriting app.py


In [13]:
from pyngrok import conf, ngrok
import subprocess
conf.get_default().auth_token = "2vqt2ViFkBfyFYhvAlLLMxqtw2b_5g2rj3bqZHxiM46567MY4"

# Disconnect semua tunnel lama
for conn in ngrok.get_tunnels():
    ngrok.disconnect(conn.public_url)

# Jalankan Streamlit
subprocess.Popen(["streamlit", "run", "app.py"])

# Tampilkan URL publik
public_url = ngrok.connect(8501)
print(f"Aplikasi dapat diakses di: {public_url}")


Aplikasi dapat diakses di: NgrokTunnel: "https://9c0e-34-125-71-212.ngrok-free.app" -> "http://localhost:8501"
